In [1]:
import os
import myServices 
import randForReg as rfr
import dask
import dask_cudf
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'osgeo'

In [ ]:
df = pd.read_csv("/Users/abdielfer/DESS/Internship2022/RNCanWork/datasetBasin1.csv")
teatDF = df[0:100,:]
df=pd.to_csv('./teatDF.csv')

In [2]:
df.head()

NameError: name 'df' is not defined

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [ ]:
#Data import and exploration
datasetPath = ''
rfReg  = rfr.RandomForestRegression(datasetPath,'porcentage')
x_train,x_validation,y_train, y_validation = rfReg.getSplitedDataset()
rfr.printDataBalace(x_train, x_validation, y_train, y_validation,'porcentage')